In [23]:
import os
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import (
    LlamaCppEmbeddings, 
    HuggingFaceEmbeddings, 
    SentenceTransformerEmbeddings
)
import faiss

import fasttext
import numpy as np

In [2]:
df1 = pd.read_csv('ramayan_data.csv')

In [3]:
df1

,text,translation,filename,source
0,Sarga Chapter- 53,Sarga Chapter- 53,kiskindhakand,RamcharitManas- Valimiki Ramayana
1,॥ श्रीसीतारामचन्द्राभ्यां नमः॥ श्रीमद्वाल्मीकी...,लौटने की अवधि बीत जाने पर भी कार्य सिद्ध न होन...,kiskindhakand,RamcharitManas- Valimiki Ramayana
2,ततस्ते ददृशुर्घोरं सागरं वरुणालयम्। अपारमभिगर्...,तदनन्तर उन श्रेष्ठ वानरों ने वरुण की निवासभूमि...,kiskindhakand,RamcharitManas- Valimiki Ramayana
3,मयस्य मायाविहितं गिरिदुर्गं विचिन्वताम्। तेषां...,मयासुर के अपनी मायाद् वारा बनाये हुए पर्वत की ...,kiskindhakand,RamcharitManas- Valimiki Ramayana
4,विन्ध्यस्य तु गिरेः पादे सम्प्रपुष्पितपादपे। उ...,"विन्ध्यगिरि के पार्श्ववर्ती पर्वत पर, जहाँ के ...",kiskindhakand,RamcharitManas- Valimiki Ramayana
...,...,...,...,...
17084,ते वयं भवता रक्ष्या भवद्विषयवासिनः। नगरस्थो वन...,"‘हम आपके राज्य में निवास करते हैं, अतः आपको हम...",aranyakanda,RamcharitManas- Valimiki Ramayana
17085,न्यस्तदण्डा वयं राजञ्जितक्रोधा जितेन्द्रियाः। ...,‘राजन्! हमने जीवमात्र को दण्ड देना छोड़ दिया ह...,aranyakanda,RamcharitManas- Valimiki Ramayana
17086,एवमुक्त्वा फलैर्मूलैः पुष्पैरन्यैश्च राघवम्। व...,ऐसा कहकर उन तपस्वी मुनियों ने वन में उत्पन्न ह...,aranyakanda,RamcharitManas- Valimiki Ramayana
17087,तथान्ये तापसाः सिद्धा रामं वैश्वानरोपमाः। न्या...,इनके सिवा दूसरे अग्नितुल्य तेजस्वी तथा न्याययु...,aranyakanda,RamcharitManas- Valimiki Ramayana


In [4]:
def embed_index(doc_list, embed_fn, index_store):
  """Function takes in existing vector_store, 
  new doc_list and embedding function that is 
  initialized on appropriate model. Local or online. 
  New embedding is merged with the existing index. If no 
  index given a new one is created"""
  #check whether the doc_list is documents, or text
  try:
    faiss_db = FAISS.from_documents(doc_list, 
                              embed_fn)  
  except Exception as e:
    faiss_db = FAISS.from_texts(doc_list, 
                              embed_fn)
  
  if os.path.exists(index_store):
    local_db = FAISS.load_local(index_store,embed_fn)
    #merging the new embedding with the existing index store
    local_db.merge_from(faiss_db)
    print("Merge completed")
    local_db.save_local(index_store)
    print("Updated index saved")
  else:
    faiss_db.save_local(folder_path=index_store)
    print("New store created...")


In [5]:
data =df1['text'].tolist()

In [6]:
data

['Sarga Chapter- 53',
 '॥ श्रीसीतारामचन्द्राभ्यां नमः॥ श्रीमद्वाल्मीकीय रामायण किष्किन्धाकाण्डम् त्रिपञ्चाशः सर्गः (सर्ग 53)',
 'ततस्ते ददृशुर्घोरं सागरं वरुणालयम्। अपारमभिगर्जन्तं घोरैरूर्मिभिराकुलम्॥१॥',
 'मयस्य मायाविहितं गिरिदुर्गं विचिन्वताम्। तेषां मासो व्यतिक्रान्तो यो राज्ञा समयः कृतः॥२॥',
 'विन्ध्यस्य तु गिरेः पादे सम्प्रपुष्पितपादपे। उपविश्य महात्मानश्चिन्तामापेदिरे तदा ॥३॥',
 'ततः पुष्पातिभाराग्राल्लताशतसमावृतान्। द्रुमान् वासन्तिकान् दृष्ट्वा बभूवुर्भयशङ्किताः॥४॥',
 'ते वसन्तमनुप्राप्तं प्रतिवेद्य परस्परम्। नष्टसंदेशकालार्था निपेतुर्धरणीतले॥५॥',
 'ततस्तान् कपिवृद्धांश्च शिष्टांश्चैव वनौकसः। वाचा मधुरयाऽऽभाष्य यथावदनुमान्य च॥६॥ स तु सिंहवृषस्कन्धः पीनायतभुजः कपिः। युवराजो महाप्राज्ञ अङ्गदो वाक्यमब्रवीत्॥७॥',
 'शासनात् कपिराजस्य वयं सर्वे विनिर्गताः। मासः पूर्णो बिलस्थानां हरयः किं न बुध्यत॥८॥ वयमाश्वयुजे मासि कालसंख्याव्यवस्थिताः। प्रस्थिताः सोऽपि चातीतः किमतः कार्यमुत्तरम्॥९॥',
 'भवन्तः प्रत्ययं प्राप्ता नीतिमार्गविशारदाः। हितेष्वभिरता भर्तुर्निसृष्टाः सर्वकर्मसु॥१०॥',
 'कर

# MURIL Embeddings 

In [7]:
embeddings1 = HuggingFaceEmbeddings(model_name="google/muril-base-cased")


No sentence-transformers model found with name /home/SaiKashyap/.cache/torch/sentence_transformers/google_muril-base-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/SaiKashyap/.cache/torch/sentence_transformers/google_muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical 

In [10]:
embed_index(doc_list=data,
            embed_fn=embeddings1,
            index_store='muril')

New store created...


# AI4BHARAT - IndicBERT V2 Embeddings 

In [11]:
embeddings2 = HuggingFaceEmbeddings(model_name="ai4bharat/IndicBERTv2-MLM-only")


No sentence-transformers model found with name /home/SaiKashyap/.cache/torch/sentence_transformers/ai4bharat_IndicBERTv2-MLM-only. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/SaiKashyap/.cache/torch/sentence_transformers/ai4bharat_IndicBERTv2-MLM-only were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exac

In [12]:
embed_index(doc_list=data,
            embed_fn=embeddings2,
            index_store='indicBERTV2')

New store created...


# FASTTEXT Embeddings 

In [18]:
def embed_index_fasttext(doc_list, index_store):
    try:
        model = fasttext.load_model('cc.sa.300.bin')  # Load your pre-trained fastText model
        faiss_db = faiss.IndexIDMap(faiss.IndexFlatIP(model.get_dimension()))
        embeddings = []
        for i, doc in enumerate(doc_list):
            embedding = model.get_sentence_vector(doc)
            embeddings.append(embedding)
            faiss_db.add_with_ids(embedding.reshape(1, -1), np.array([i]))
    except Exception as e:
        raise ValueError("Embedding failed: " + str(e))

    if os.path.exists(index_store):
        local_db = faiss.read_index(index_store)
        local_db.add(embeddings)
        faiss.write_index(local_db, index_store)
        print("Merge completed")
        print("Updated index saved")
    else:
        faiss.write_index(faiss_db, index_store)
        print("New store created...")




In [24]:
embed_index_fasttext(data, 'fasttext1')

New store created...


# Compare Semantic Search 

In [27]:
docsearch1 = FAISS.load_local("muril",embeddings1)
docsearch2 = FAISS.load_local("indicBERTV2", embeddings2)

# Using MURIL

In [35]:
query = "गुरुरेष महाहुः"
docs1 = docsearch1.similarity_search(query)
page_content1 = [doc.page_content for doc in docs1]

print(page_content1)

[' ', 'श्रोतव्यं सर्वमेतस्य भवद्भिर्दिष्टकारिभिः। गुरुरेष महाबाहुः श्वशुरो मे महाबलः॥५४॥', 'सिताभ्रशिखरप्रख्यं प्रासादमधिरुह्य च। समीयाय नरेन्द्रेण शक्रेणेव बृहस्पतिः॥२२॥', 'उद्दिष्टं दीर्घमायुस्ते दैवज्ञैरपि राघव। अनृतं वचनं तेषामल्पायुरसि राघव॥१२॥']


# Using IndicBERTv2

In [36]:
docs2 = docsearch2.similarity_search(query)
page_content2 = [doc.page_content for doc in docs2]

print(page_content2)

['एष वञ्जुलको नाम पक्षी परमदारुणः। आवयोर्विजयं यद्धे शंसन्निव विनर्दति॥२३॥', 'नलश्चक्रे महासेतुं मध्ये नदनदीपतेः। स तदा क्रियते सेतुर्वानोरकर्मभिः॥६३॥', 'जीवितान्तकरं घोरं सादृश्यात् तु विशङ्कितः। मूलघातो न नौ स्याद्धि द्वयोरिति कृतो मया॥ ३३॥', 'व्यसनी वा समृद्धो वा गतिरेष तवानघ। एष लोके सतां धर्मो यज्ज्येष्ठवशगो भवेत्॥६॥']


# Using Fasttext

In [58]:
model1 = fasttext.load_model('cc.sa.300.bin')  # Load your pre-trained fastText model
index1 = faiss.read_index('fasttext1')


In [60]:
def searchQueryInFaiss(query,index,top_k):
  D, I = index.search(np.array([model1.get_sentence_vector(query)]), k=top_k)
  similarityScores = D.flatten().tolist()
  topIndexes = I.flatten().tolist()
  return similarityScores,topIndexes

In [63]:
def IndexToText(topIndexes,allQuestions):
  retData= []
  for eachTopIndex in topIndexes:
    retData.append(allQuestions[eachTopIndex])

  return retData
     

In [61]:
similarityScores,topIndexes = searchQueryInFaiss(query,index1,5)


In [62]:
topIndexes

[13258, 16931, 2443, 4885, 2731]

In [64]:
similar = IndexToText(topIndexes,data)

In [65]:
similar

['मन्थराप्रभवस्तीव्र कैकेयीग्राहसंकुलः। वरदानमयोऽक्षोभ्योऽमज्जयच्छोकसागरः॥१३॥',
 'रोमभिर्निशितैस्तीक्ष्णैर्महागिरिमिवोच्छ्रितम्। नीलमेघनिभं रौद्रं मेघस्तनितनिःस्वनम्॥२८॥',
 'सिंहशार्दूलसहितं मत्तमातङ्गसेवितम्। मत्तद्विजगणोद्घष्टं सलिलोत्पीडसंकुलम्॥४१॥',
 'विसृजन्तो महानादांस्त्रासयन्तो निशाचरान्। लङ्कादाराण्युपाजग्मुर्योद्धुकामाः प्लवंगमाः॥६४॥',
 'निविष्टं निविशच्चैव भीमनादं महाबलम्। तबलार्णवमक्षोभ्यं ददृशाते निशाचरौ॥१२॥']

# Some Issue with inltk & Albert -base_ Sanskrit.. 

vectors = get_embedding_vectors('भारत', 'sa')
